In [7]:
import pandas as pd
from shapely import wkt
from shapely.geometry import mapping
import json

# Load the Excel file
file_path = '/Users/fer/Documents/PYTHON/11_coastalDroneMetadataMap_GitHub/raw_data/coastalDroneMetadataMap.xlsx'
data = pd.read_excel(file_path)
# Print the head of the dataframe
data.head()

,dwc:eventID,dwc:eventRemarks,dwc:waterBody,dwc:locality,dwc:decimalLatitude,dwc:decimalLongitude,dwc:footprintWKT,dwc:geodeticDatum,dwc:institutionCode,dwc:license,...,drone,sensor,images,altitude,area,rep_error,duration,GSD,GCP,groundtruth
0,20230925_090700_CATGRASS23_CREUS_RUSTELLA,CATGRASS23,Mediterranean,CREUS_RUSTELLA,42.242961,3.227340,"POLYGON ((3.224813213464021 42.244891287469, 3...",EPSG:4326,GEAM IEO-CSIC,https://creativecommons.org/licenses/by-nc/4.0/,...,MAVIC 2,RGB,212,87.9,0.0943,3.28,NaN,2.07,YES,NO
1,20230925_095100_CATGRASS23_CREUS_MONTJOI,CATGRASS23,Mediterranean,CREUS_MONTJOI,42.249427,3.229456,POLYGON ((3.225349602255836 42.252491761185134...,EPSG:4326,GEAM IEO-CSIC,https://creativecommons.org/licenses/by-nc/4.0/,...,MAVIC 2,RGB,306,106.0,0.2330,3.31,NaN,2.48,YES,NO
2,20230925_110000_CATGRASS23_CREUS_PELOSA,CATGRASS23,Mediterranean,CREUS_PELOSA,42.249765,3.240474,"POLYGON ((3.237486233542744 42.25119347924394,...",EPSG:4326,GEAM IEO-CSIC,https://creativecommons.org/licenses/by-nc/4.0/,...,MAVIC 2,RGB,221,101.0,0.0935,4.54,NaN,2.38,YES,NO
3,20230925_141200_CATGRASS23_CREUS_JUGADORA,CATGRASS23,Mediterranean,CREUS_JUGADORA,42.316508,3.312677,"POLYGON ((3.310410482574943 42.31827884442748,...",EPSG:4326,GEAM IEO-CSIC,https://creativecommons.org/licenses/by-nc/4.0/,...,MAVIC 2,RGB,244,82.5,0.0798,2.33,NaN,1.94,YES,NO
4,20240124_091200_NORGRASS24_RIBADESELLA,NORGRASS24,Atlantic,RIBADESELLA,43.458248,-5.063837,POLYGON ((-5.067583709182484 43.46337398396899...,EPSG:4326,GEAM IEO-CSIC,https://creativecommons.org/licenses/by-nc/4.0/,...,MAVIC 3M,RGB+MS,5970,92.5,0.5260,0.25,71.0,2.48,NO,NO


In [8]:
# Select relevant columns for conversion to JSON
relevant_columns = ['dwc:eventID', 'dwc:eventDate', 'dwc:institutionCode', 'dwc:decimalLatitude', 'dwc:decimalLongitude', 'dwc:footprintWKT', 'contact', 'funding' ]

# Ensure the columns are in the correct format for Leaflet
data_filtered = data[relevant_columns].copy()

# Convert eventDate to string
data_filtered['dwc:eventDate'] = data_filtered['dwc:eventDate'].astype(str)
data_filtered.head()

# Rename columns to match the JSON structure
data_filtered.rename(columns={
    'dwc:eventID': 'eventID',
    'dwc:eventDate': 'eventDate',
    'dwc:institutionCode': 'institutionCode',
    'dwc:decimalLatitude': 'decimalLatitude',
    'dwc:decimalLongitude': 'decimalLongitude',
    'dwc:footprintWKT': 'footprintWKT'
}, inplace=True)



In [9]:
# Convert footprintWKT to list of coordinate pairs for polygons (Leaflet expects [latitude, longitude])
def convert_wkt_to_coordinates(wkt_str):
    geom = wkt.loads(wkt_str)
    return [(coord[1], coord[0]) for coord in geom.exterior.coords]

data_filtered['polygon'] = data_filtered['footprintWKT'].apply(convert_wkt_to_coordinates)

# Convert decimalLatitude and decimalLongitude to points
def convert_to_point(lat, lon):
    return (lat, lon)

data_filtered['centroid'] = data_filtered.apply(lambda row: convert_to_point(row['decimalLatitude'], row['decimalLongitude']), axis=1)


In [10]:
# Convert the DataFrame to a JSON-like structure
json_data = data_filtered.to_dict(orient='records')

# Prepare the JavaScript variable with data
js_variable = f"var data = {json.dumps(json_data, indent=4)};"

# Save to a .js file
output_js_path = '/Users/fer/Documents/PYTHON/11_coastalDroneMetadataMap_GitHub/docs/data/coastalDroneMetadataMap.js'
with open(output_js_path, 'w') as js_file:
    js_file.write(js_variable)


# Print the path to the JSON file
print(f"JSON file saved to: {output_js_path}")


JSON file saved to: /Users/fer/Documents/PYTHON/11_coastalDroneMetadataMap_GitHub/docs/data/coastalDroneMetadataMap.js


In [11]:
# Create GeoJSON for flight polygons
polygons = {
    "type": "FeatureCollection",
    "features": []
}

for _, row in data_filtered.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [row['polygon']]
        },
        "properties": {
            "eventID": row['eventID'],
            "eventDate": row['eventDate'],
            "institutionCode": row['institutionCode'],
            "contact": row['contact'],
            "funding": row['funding']
        }
    }
    polygons["features"].append(feature)

# Save flight polygons to a GeoJSON file
polygons_path = '/Users/fer/Documents/PYTHON/11_coastalDroneMetadataMap_GitHub/docs/data/flight_polygons.geojson'
with open(polygons_path, 'w') as geojson_file:
    json.dump(polygons, geojson_file, indent=4)
    print(f"GEOJSON saved as {polygons_path}")

GEOJSON saved as /Users/fer/Documents/PYTHON/11_coastalDroneMetadataMap_GitHub/docs/data/flight_polygons.geojson


In [12]:
# Create GeoJSON for flight centroids
centroids = {
    "type": "FeatureCollection",
    "features": []
}

for _, row in data_filtered.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": row['centroid']
        },
        "properties": {
            "eventID": row['eventID'],
            "eventDate": row['eventDate'],
            "institutionCode": row['institutionCode'],
            "contact": row['contact'],
            "funding": row['funding']
        }
    }
    centroids["features"].append(feature)

# Save flight centroids to a GeoJSON file
centroids_path = '/Users/fer/Documents/PYTHON/11_coastalDroneMetadataMap_GitHub/docs/data/flight_centroids.geojson'
with open(centroids_path, 'w') as geojson_file:
    json.dump(centroids, geojson_file, indent=4)
    print(f"GEOSON saved as {centroids_path}")

GEOSON saved as /Users/fer/Documents/PYTHON/11_coastalDroneMetadataMap_GitHub/docs/data/flight_centroids.geojson
